In [1]:
import numpy as np

### Task 1. Sigmoid

Напишем forward и backward pass для функции $$f(x, w) = \frac{1}{1 + \exp(-(w_0 x + w_1))}$$

#### Forward pass

Пусть $x = 2, w_0 = 3, w_1 = -4$

In [2]:
x = 2
w = [3, -4]

f = 1 / (1 + np.exp(-(x * w[0] + w[1])))

print('f = {}'.format(f))

f = 0.8807970779778823


Теперь разобьем вычисление на части, чтобы было удобнее считать производные

In [3]:
a = x * w[0]
b = a + w[1]
c = -1 * b
d = np.exp(c)
e = 1 + d
f = 1 / e

print('f = {}'.format(f))

f = 0.8807970779778823


#### Backward pass

Для упрощения имен переменных здесь и далее в коде будем придерживаться следующего алиаса: $$\frac{df}{dx} = dx$$

In [4]:
df = 1
de = -1 / e**2
dd = de * 1
dc = dd * np.exp(c)
db = dc * -1
da = db * 1

dx = da * w[0]
dw0 = da * x
dw1 = db * 1

print('dx = {}\n'
      'dw0 = {}\n'
      'dw1 = {}'.
      format(dx, dw0, dw1))

dx = 0.3149807562105195
dw0 = 0.209987170807013
dw1 = 0.1049935854035065


Проверим производную $\frac{df}{db}$ воспользовавшись знанием, что $$f = \sigma(b)$$ $$\frac{d\sigma(x)}{dx} = \sigma(x)(1 - \sigma(x))$$

In [5]:
def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

dsigmoid = sigmoid(b) * (1 - sigmoid(b))

print('dsigmoid = \t{}\n'
      'db = \t\t{}\n'.
      format(dsigmoid, db))
assert np.isclose(dsigmoid, db)

dsigmoid = 	0.10499358540350662
db = 		0.1049935854035065



#### Gradient checking

А теперь проверим, что мы правильно посчитали все производные

Для этого воспользуемся [gradient checking](http://ufldl.stanford.edu/tutorial/supervised/DebuggingGradientChecking/).

Суть этого метода заключается в численном вычислении аппроксимации производной по формуле $$\frac{df}{dx} = \frac{f(x + \epsilon) - f(x - \epsilon)}{2\epsilon}$$

Сначала проверим, что мы верно нашли производную $\sigma(b)$

In [6]:
print('sigmoid(b) = {}'.format(sigmoid(b)))

sigmoid(b) = 0.8807970779778823


In [7]:
EPS = 1e-5

dsigmoid_check = (sigmoid(b + EPS) - sigmoid(b - EPS)) / (2 * EPS)

print('dsigmoid = \t\t{}\n'
      'dsigmoid_check = \t{}\n'.
      format(dsigmoid, dsigmoid_check))
assert np.isclose(dsigmoid, dsigmoid_check)

dsigmoid = 		0.10499358540350662
dsigmoid_check = 	0.10499358540916325



А теперь проверим производные нашей функции $f$

In [8]:
def calculate_f(x, w):
    return sigmoid(x * w[0] + w[1])

f = calculate_f(x, w)

print('f = {}'.format(f))

f = 0.8807970779778823


In [9]:
dx_check = (calculate_f(x + EPS, w) - calculate_f(x - EPS, w)) / (2 * EPS)

print('dx = \t\t{}\n'
      'dx_check = \t{}\n'.
      format(dx, dx_check))
assert np.isclose(dx, dx_check)

dw0_check = (calculate_f(x, [w[0] + EPS, w[1]]) - calculate_f(x, [w[0] - EPS, w[1]])) / (2 * EPS)

print('dw0 = \t\t{}\n'
      'dw0_check = \t{}\n'.
      format(dw0, dw0_check))
assert np.isclose(dw0, dw0_check)

dw1_check = (calculate_f(x, [w[0], w[1] + EPS]) - calculate_f(x, [w[0], w[1] - EPS])) / (2 * EPS)

print('dw1 = \t\t{}\n'
      'dw1_check = \t{}\n'.
      format(dw1, dw1_check))
assert np.isclose(dw1, dw1_check)

dx = 		0.3149807562105195
dx_check = 	0.3149807562330409

dw0 = 		0.209987170807013
dw0_check = 	0.2099871708183265

dw1 = 		0.1049935854035065
dw1_check = 	0.10499358540916325



### Task 2. Матрицы

Реализуем forward и backward pass для функции $$L = \lVert XW - Y \rVert_2^2 + \lambda \lVert W \rVert_1 $$

<img src="img/example2.jpg">

In [10]:
np.random.seed(42 * 2)

X = np.random.randint(-4, 4, size=(3, 2))
W = np.random.randint(-4, 4, size=(2, 1))
Y = np.random.randint(-4, 4, size=(3, 1))
lam = 2 * (-1 if np.random.randint(2) == 0 else 1)

print('X = {}\n'
      'W = {}\n'
      'Y = {}\n'
      'lambda = {}\n'.
      format(X, W, Y, lam))

X = [[-2 -3]
 [-3 -4]
 [ 1  0]]
W = [[2]
 [1]]
Y = [[-4]
 [ 2]
 [-3]]
lambda = -2



In [4]:
X = np.array([[0], [-2], [1]], dtype=float)
Y = np.array([[0, 1], [1, -1], [0, -2]], dtype=float)
W = np.array([[1, 1]], dtype=float)
lam = 1.0

#### Forward pass

In [5]:
pred = X.dot(W)
pred_loss = np.sum((pred - Y) ** 2)
reg_loss = np.sum(np.abs(W))
loss = pred_loss + lam * reg_loss
loss

23.0

#### Backward pass

In [9]:
np.sign(W)

array([[1., 1.]])

In [6]:
dloss = 1
dreg_loss = lam * dloss
dpred_loss = dloss
dW1 = np.sign(W) * dreg_loss

dpred = dpred_loss * 2 * (pred - Y)
dW2 = X.T.dot(dpred)

dW = dW1 + dW2
print(dW)

[[15. 11.]]


#### Gradient checking

In [13]:
def calculate_loss(X, W, Y, lam):
    pred = X.dot(W)
    pred_loss = np.sum((pred - Y)**2)
    reg_loss = np.sum(np.abs(W))
    loss = pred_loss + lam * reg_loss
    return loss

calculate_loss(X, W, Y, lam)

172

In [14]:
EPS = 1e-5
dW_check = np.zeros_like(dW, dtype=np.float32)

for i in range(dW.shape[0]):
    for j in range(dW.shape[1]):
        eps = np.zeros_like(W, dtype=np.float32)
        eps[i, j] = EPS
        W_add_eps = W + eps       
        W_sub_eps = W - eps
        
        dW_check[i, j] = (calculate_loss(X, W_add_eps, Y, lam) - calculate_loss(X, W_sub_eps, Y, lam)) / (2 * float(EPS))

        
print('dW = \t\t{}\n'
      'dW_check = \t{}\n'.
      format(dW, dW_check))

dW = 		[[ 92]
 [112]]
dW_check = 	[[ 92.]
 [112.]]



### Task 3. Simple NN

<img src="img/example3.jpg">

In [15]:
from collections import OrderedDict
from copy import copy

import numpy as np


def relu(x):
    return np.maximum(x, 0)

def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

def fprop(x, y, params):
    W1, b1, W2, b2 = [params[key] for key in ('W1', 'b1', 'W2', 'b2')]
    z1 = np.dot(W1, x) + b1
    a1 = relu(z1)
    z2 = np.dot(W2, a1) + b2
    a2 = sigmoid(z2)
    loss = -(y * np.log(a2) + (1 - y) * np.log(1 - a2))
    
    ret = OrderedDict({'x': x, 'y': y, 'z1': z1, 'a1': a1, 'z2': z2, 'a2': a2, 'loss': loss})
    for key in params:
        ret[key] = params[key]
    return ret

def bprop(fprop_cache):
    x, y, z1, a1, z2, a2, loss = [fprop_cache[key] for key in ('x', 'y', 'z1', 'a1', 'z2', 'a2', 'loss')]
    
    dz2 = (a2 - y)
    dW2 = np.dot(dz2, a1.T)
    db2 = dz2
        
    da1 = np.dot(fprop_cache['W2'].T, dz2)
    dz1 = da1 * (z1 > 0)
    dW1 = np.dot(dz1, x.T)
    db1 = dz1
    return OrderedDict({'W1': dW1, 'b1': db1, 'W2': dW2, 'b2': db2})

In [16]:
np.random.seed(42)

W1 = np.random.rand(2, 2)
b1 = np.random.rand(2, 1)
W2 = np.random.rand(1, 2)
b2 = np.random.rand(1, 1)
params = {'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2}
x = np.random.rand(2, 1)
y = np.random.randint(0, 2)

print('x = ', x)
print('y = ', y)

x =  [[0.70807258]
 [0.02058449]]
y =  1


In [17]:
fprop_cache = fprop(x, y, params)
print('loss')
print(fprop_cache['loss'])

bprop_cache = bprop(fprop_cache)

for key in bprop_cache:
    print('d' + key)
    print(bprop_cache[key])

loss
[[0.25835725]]
dW1
[[-0.00936392 -0.00027222]
 [-0.13964014 -0.0040595 ]]
db1
[[-0.01322452]
 [-0.19721162]]
dW2
[[-0.10035944 -0.1563307 ]]
db2
[[-0.22768073]]


In [18]:
eps = 1e-6
check_cache = OrderedDict()

# For every single parameter (W, b)
for key in params:
    param = params[key]
    param_check_cache = np.zeros(param.shape)
    for j in range(param_check_cache.shape[0]):
        for k in range(param_check_cache.shape[1]):
            # For every element of parameter matrix, compute gradient of loss wrt
            # that element numerically using finite differences
            param_add_eps = np.copy(param)
            param_sub_eps = np.copy(param)
            param_add_eps[j, k] += eps
            param_sub_eps[j, k] -= eps
            
            add_params = copy(params)
            sub_params = copy(params)
            add_params[key] = param_add_eps
            sub_params[key] = param_sub_eps

            param_check_cache[j, k] = (fprop(x, y, add_params)['loss'] - fprop(x, y, sub_params)['loss']) / (2 * eps)
    check_cache[key] = param_check_cache

for key in params:
    print('d{} = \n{}'.format(key, bprop_cache[key]))
    print('d{}_check = \n{}'.format(key, check_cache[key]))
    print('===================')

    assert np.isclose(bprop_cache[key], check_cache[key]).all()

dW1 = 
[[-0.00936392 -0.00027222]
 [-0.13964014 -0.0040595 ]]
dW1_check = 
[[-0.00936392 -0.00027222]
 [-0.13964014 -0.0040595 ]]
db1 = 
[[-0.01322452]
 [-0.19721162]]
db1_check = 
[[-0.01322452]
 [-0.19721162]]
dW2 = 
[[-0.10035944 -0.1563307 ]]
dW2_check = 
[[-0.10035944 -0.1563307 ]]
db2 = 
[[-0.22768073]]
db2_check = 
[[-0.22768073]]


### Task 4. Softmax

In [19]:
def softmax(x):
    return np.exp(x) / (np.sum(np.exp(x), axis=0, keepdims=True) + EPS)

def softmax_grad(x):
    # Reshape the 1-d softmax to 2-d so that np.dot will do the matrix multiplication
    x = x.reshape(-1, 1)
    return np.diagflat(x) - np.dot(x, x.T)

In [20]:
x = np.array([1, 2])
s = softmax(x)
s

array([0.26894116, 0.73105786])

In [21]:
dx = softmax_grad(softmax(x))
dx

array([[ 0.19661181, -0.19661154],
       [-0.19661154,  0.19661227]])

In [22]:
EPS = 1e-4
dx_check = np.zeros_like(dx, dtype=np.float32)

for i in range(dx.shape[0]):
    eps = np.zeros_like(x, dtype=np.float32)
    eps[i] = EPS
    x_add = x + eps
    x_sub = x - eps

    dx_check[i] = (softmax(x_add) - softmax(x_sub)) / (2 * float(EPS))

        
print('dx = \n{}'.format(dx))
print('dx_check = \n{}'.format(dx_check))

dx = 
[[ 0.19661181 -0.19661154]
 [-0.19661154  0.19661227]]
dx_check = 
[[ 0.1966107  -0.19660804]
 [-0.19660804  0.19661526]]
